In [ ]:
!pip install -q transformers>=4.45.0 timm accelerate bitsandbytes
!pip install -q tensorflow-datasets tensorflow-hub
!pip install -q -U bitsandbytes

In [ ]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor, AutoConfig
from PIL import Image
import tensorflow_datasets as tfds
import numpy as np
import sys
import os

In [ ]:
# Loading the SOTA Model Weights from Hugging Face
model_id = "openvla/openvla-7b"
save_directory = "./vla_model_local"

processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/130 [00:00<?, ?B/s]

processing_prismatic.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/openvla/openvla-7b:
- processing_prismatic.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


preprocessor_config.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

In [ ]:
from transformers.models.auto.modeling_auto import AutoModelForVision2Seq
config = AutoConfig.from_pretrained(model_id, trust_remote_code=True)

try:
    vla = AutoModelForVision2Seq.from_config(config, trust_remote_code=True)
except Exception:
    pass

# Applying the patch globally to the remote class
for module_name, module in sys.modules.items():
    if "modeling_prismatic" in module_name:
        if hasattr(module, "OpenVLAForActionPrediction"):
            setattr(module.OpenVLAForActionPrediction, "_supports_sdpa", True)
            print(f"✅ Globally patched: {module_name}")

config.json: 0.00B [00:00, ?B/s]

configuration_prismatic.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/openvla/openvla-7b:
- configuration_prismatic.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/usr/local/lib/python3.12/dist-packages/transformers/models/auto/modeling_auto.py:2275: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


modeling_prismatic.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/openvla/openvla-7b:
- modeling_prismatic.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


✅ Globally patched: transformers_modules.openvla.openvla_hyphen_7b.31f090d05236101ebfc381b61c674dd4746d4ce0.modeling_prismatic


In [ ]:
if os.path.exists(save_directory) and len(os.listdir(save_directory)) > 0:
    print("Found local weights! Loading from disk (No download needed)...")
    load_path = save_directory
else:
    print("Local weights not found. Downloading shards (This will take 10+ mins)...")
    load_path = model_id

vla = AutoModelForVision2Seq.from_pretrained(
    load_path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    load_in_4bit=True,
    attn_implementation="sdpa",
    resume_download=True,
    device_map="auto"
).to("cuda")

if load_path == model_id:
    print("Saving model to local disk for future use...")
    vla.save_pretrained(save_directory)
    processor.save_pretrained(save_directory)
    print(f"✅ Model saved to {save_directory}. Future errors won't require a re-download.")

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/modeling_auto.py:2284: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Local weights not found. Downloading shards (This will take 10+ mins)...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/6.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

Saving model to local disk for future use...
✅ Model saved to ./vla_model_local. Future errors won't require a re-download.


In [ ]:
!pip uninstall -y timm
!pip install -q "timm==0.9.12"

Found existing installation: timm 1.0.24
Uninstalling timm-1.0.24:
  Successfully uninstalled timm-1.0.24
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 34.7 MB/s eta 0:00:00


In [ ]:
import timm
print(timm.__version__)

0.9.12


In [ ]:
import numpy as np
import torch
from PIL import Image

In [ ]:
vla.eval()

OpenVLAForActionPrediction(
  (vision_backbone): PrismaticVisionBackbone(
    (featurizer): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear4bit(in_features=1024, out_features=3072, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): LayerScale()
          (drop_path1): Identity()
          (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
       

In [ ]:
!pip install -q apache_beam tfds-nightly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 27.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires grpcio>=1.71.2, but you have grpcio 1.65.5 which is incompatible.


In [ ]:
# Downloading the first shard (approx 100MB)
!wget https://rail.eecs.berkeley.edu/datasets/bridge_release/data/tfds/bridge_dataset/1.0.0/bridge_dataset-train.tfrecord-00000-of-01024

--2026-01-18 17:23:00--  https://rail.eecs.berkeley.edu/datasets/bridge_release/data/tfds/bridge_dataset/1.0.0/bridge_dataset-train.tfrecord-00000-of-01024
Resolving rail.eecs.berkeley.edu (rail.eecs.berkeley.edu)... 128.32.244.190
Connecting to rail.eecs.berkeley.edu (rail.eecs.berkeley.edu)|128.32.244.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 108381239 (103M)
Saving to: ‘bridge_dataset-train.tfrecord-00000-of-01024’

bridge_dataset-trai 100%[===================>] 103.36M  41.2MB/s    in 2.5s    

2026-01-18 17:23:03 (41.2 MB/s) - ‘bridge_dataset-train.tfrecord-00000-of-01024’ saved [108381239/108381239]



In [ ]:
import tensorflow as tf

# Path to the shard of data.
shard_path = "bridge_dataset-train.tfrecord-00000-of-01024"

In [ ]:
# Defining the 'RLDS' structure for Bridge V2
# This matches the schema found at the URL
def parse_fn(example_proto):
    features = {
        'steps/observation/image': tf.io.FixedLenSequenceFeature([], tf.string, allow_missing=True),
        'steps/observation/natural_language_instruction': tf.io.FixedLenSequenceFeature([], tf.string, allow_missing=True),
        'steps/action': tf.io.FixedLenSequenceFeature([7], tf.float32, allow_missing=True),
    }
    return tf.io.parse_single_example(example_proto, features)

In [ ]:
raw_dataset = tf.data.TFRecordDataset(shard_path)
parsed_dataset = raw_dataset.map(parse_fn)

In [ ]:
raw_example = next(iter(tf.data.TFRecordDataset(shard_path)))
example = tf.train.Example.FromString(raw_example.numpy())
print("Keys available in this shard:", example.features.feature.keys())

In [ ]:
shard_path = "bridge_dataset-train.tfrecord-00000-of-01024"

def parse_fn_bridge_v1(example_proto):
    features = {
        'steps/language_instruction': tf.io.FixedLenSequenceFeature([], tf.string, allow_missing=True),
        'steps/observation/image_2': tf.io.FixedLenSequenceFeature([], tf.string, allow_missing=True),
        'steps/action': tf.io.FixedLenSequenceFeature([7], tf.float32, allow_missing=True),
    }
    return tf.io.parse_single_example(example_proto, features)

raw_dataset = tf.data.TFRecordDataset(shard_path)
parsed_dataset = raw_dataset.map(parse_fn_bridge_v1)

for episode in parsed_dataset.take(100):
    instructions = episode['steps/language_instruction'].numpy()

    if len(instructions) > 0:
        instruction = instructions[0].decode('utf-8')

        if "pick" in instruction.lower():
            img_seq = episode['steps/observation/image_2'].numpy()

            if len(img_seq) > 0:
                image_pil = Image.fromarray(tf.io.decode_image(img_seq[0]).numpy())

                inputs = processor(text=instruction, images=image_pil, return_tensors="pt").to("cuda")
                inputs_dict = {k: v for k, v in inputs.items()}

                with torch.no_grad():
                    # Unpacking dict to provide individual tensors to the model
                    predicted_action = vla.predict_action(**inputs_dict, unnorm_key="bridge_orig")

                gt_action = episode['steps/action'].numpy()[0]

                print(f"🎯 Task: {instruction}")
                print(f"Predicted XYZ: {predicted_action[:3]}")
                print(f"Ground Truth XYZ: {gt_action[:3]}")
                print(f"MSE: {np.mean((predicted_action[:3] - gt_action[:3])**2):.6f}")
                break

🎯 Task: Picked the piece of chocolate and put it into the drawer
Predicted XYZ: [-0.00020879 -0.00042412  0.00703386]
Ground Truth XYZ: [0.0000000e+00 0.0000000e+00 1.3877788e-17]
MSE: 0.000017
